In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [3]:
#loading the data

In [4]:
review=pd.read_csv("Reviews.csv")

In [5]:
review.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [5]:
review=review[review["Score"]!=3]
def partition(x):
    if x > 3:
        return 1
    return 0
review['Score']=review.Score.map(partition)

In [6]:
review = review.drop_duplicates(subset={"UserId","ProfileName","Time","Text"})
review = review[review['HelpfulnessNumerator'] <= review['HelpfulnessDenominator']]
review.shape

(364171, 10)

In [7]:
df = review.loc[40000:90000,['Summary','Score']]

In [8]:
df.head

<bound method NDFrame.head of                                          Summary  Score
40000                                Oh yeah!!!!      1
40001  Great, but not for an air popper for sure      1
40002                          Kid Safe Popcorn!      1
40003                                      Yummy      1
40004                                Not so good      0
...                                          ...    ...
89996                     The dogs rate it *****      1
89997                                    Wedding      1
89998                                  Delicious      1
89999              Liked these better as a kid..      1
90000                               Disappointed      0

[39207 rows x 2 columns]>

In [9]:
import nltk,sys 
import nltk.data
from nltk.stem.porter import *
pd.options.mode.chained_assignment = None
#from spellchecker import SpellChecker
#https://emojipedia.org/people
from textblob import TextBlob
# step 1 stop word removal
from nltk.corpus import stopwords
stop = stopwords.words("english")
#print(['Summary'])

print('-------Removing Stop Word--------')
df['Step1_SentimentText'] = df['Summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# step 2 replace special char and replace abbreviations
import csv,re
def translator(user_string):
    user_string = user_string.split(" ")
    j = 0
    for _str in user_string:
        # File path which consists of Abbreviations.
        fileName = "slang.txt"

        # File Access mode [Read Mode]
        with open(fileName, "r") as myCSVfile:
            # Reading file as CSV with delimiter as "=", so that abbreviation are stored in row[0] and phrases in row[1]
            dataFromFile = csv.reader(myCSVfile, delimiter="=")
            # Removing Special Characters.
            _str = re.sub('[^a-zA-Z0-9]+', '', _str)
            for row in dataFromFile:
                # Check if selected word matches short forms[LHS] in text file.
                if _str.upper() == row[0]:
                    # If match found replace it with its appropriate phrase in text file.
                    user_string[j] = row[1]
            myCSVfile.close()
        j = j + 1
    return ' '.join(user_string)
print('-------Replace Abbreviations--------')
df['Step2_SentimentText'] = df['Step1_SentimentText'].apply(lambda x:  translator(x)  ) 

# step 3 stemming 
ps = PorterStemmer()
print('-------Stemming--------')
df['Step3_SentimentText'] = df['Step2_SentimentText'].apply(lambda x: ' '.join([ps.stem(word) for word in x.split() ]))

# step 4 Lemmazation
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
print('-------Lemmazation--------')
df['Step4_SentimentText'] = df['Step2_SentimentText'].apply(lambda x: ' '.join([lmtzr.lemmatize(word,'v') for word in x.split() ]))
# step 6 Capitalization
print('-------Capitalization--------')
df['Step6_SentimentText'] = df['Step2_SentimentText'].apply(  lambda x: ' '.join( [ word.upper() for word in x.split() ] ) )


-------Removing Stop Word--------
-------Replace Abbreviations--------
-------Stemming--------
-------Lemmazation--------
-------Capitalization--------


In [12]:
df.head()

,Summary,Score,Step1_SentimentText,Step2_SentimentText,Step3_SentimentText,Step4_SentimentText,Step6_SentimentText
40000,Oh yeah!!!!,1,Oh yeah!!!!,Oh yeah!!!!,Oh yeah!!!!,Oh yeah!!!!,OH YEAH!!!!
40001,"Great, but not for an air popper for sure",1,"Great, air popper sure","Great, air popper sure","great, air popper sure","Great, air popper sure","GREAT, AIR POPPER SURE"
40002,Kid Safe Popcorn!,1,Kid Safe Popcorn!,Kid Safe Popcorn!,kid safe popcorn!,Kid Safe Popcorn!,KID SAFE POPCORN!
40003,Yummy,1,Yummy,Yummy,yummi,Yummy,YUMMY
40004,Not so good,0,Not good,Not good,not good,Not good,NOT GOOD


In [13]:
X = df['Step6_SentimentText']
Y = df['Score']

In [14]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size=0.2, random_state=101)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(xtrain)
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [16]:
x_train_vect = vect.transform(xtrain)
x_train_vect

<31365x8907 sparse matrix of type '<class 'numpy.int64'>'
	with 99822 stored elements in Compressed Sparse Row format>

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer().fit(xtrain)

In [18]:
vect = CountVectorizer(ngram_range = (1,2)).fit(xtrain)
x_train_vect = vect.transform(xtrain)

In [19]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(x_train_vect,ytrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [20]:
from sklearn.metrics import roc_auc_score
predictions = model.predict(vect.transform(xtest))
print('AUC: ',roc_auc_score(ytest,predictions))

AUC:  0.7335454066030865


In [21]:
from sklearn.linear_model import LogisticRegression
model1 = LogisticRegression()
model1.fit(x_train_vect,ytrain)

D:\New folder\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [22]:
predictions1 = model1.predict(vect.transform(xtest))
print('AUC: ',roc_auc_score(ytest,predictions1))

AUC:  0.775934142991537


In [23]:
print(model1.predict(vect.transform(['i m gr8']))) #great

[1]


In [24]:
print(model1.predict(vect.transform(['worst product ever'])))

[0]


In [25]:
print(model1.predict(vect.transform([': )'])))  #posit-ive --1 or neg -- 0

[1]


In [26]:
from textblob import TextBlob
sent = input()
sent_textblob = TextBlob(sent)
sent_correct = sent_textblob.correct()
sent_correct = translator(sent_correct)
print("did you mean sent {} if yes then 1 else 0".format(sent_correct))
n = int(input())
if n:
    print(model.predict(vect.transform([sent_correct])))
else:
    print(model.predict(vect.transform([sent])))

verry baad
did you mean sent very bad if yes then 1 else 0
1
[0]
